In [1]:
#import packages
import requests, sys
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
#getting wiki page as text object
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
#interpreting html with BeautifulSoup
soup = BeautifulSoup(page)
soup.prettify();

In [3]:
#filter for the first table
table = soup.findAll("table")[0]

In [4]:
#html table to pandas dataframe
df = pd.read_html(str(table), header = 0)[0]

In [5]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289 entries, 0 to 288
Data columns (total 3 columns):
Postcode         289 non-null object
Borough          289 non-null object
Neighbourhood    289 non-null object
dtypes: object(3)
memory usage: 6.9+ KB


In [7]:
dfclean = df.query("Borough != 'Not assigned'") #keep only the ones that have an assigned Borough

In [8]:
#replace non assigned neighbourhoods with the values from the Borough column
dfclean.loc[dfclean['Neighbourhood'] == "Not assigned" , "Neighbourhood"] = dfclean.loc[dfclean['Neighbourhood'] == "Not assigned" , "Borough"]

C:\Users\Johannes\AppData\Local\conda\conda\envs\mypython3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\Johannes\AppData\Local\conda\conda\envs\mypython3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
#grouping the Borgoughs and putting the neighbourhoods in lists
dfgrouped = dfclean.groupby(['Borough', 'Postcode'], as_index = False).agg({'Neighbourhood': lambda x: ', '.join(set(x))})

In [10]:
dfgrouped.head()

,Borough,Postcode,Neighbourhood
0,Central Toronto,M4N,Lawrence Park
1,Central Toronto,M4P,Davisville North
2,Central Toronto,M4R,North Toronto West
3,Central Toronto,M4S,Davisville
4,Central Toronto,M4T,"Moore Park, Summerhill East"


In [11]:
dfgrouped.shape

(103, 3)

In [21]:
import geocoder # import geocoder

In [19]:
def get_geo(postal_code):
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude, longitude

In [39]:
#get_geo('M4N')

In [35]:
#read the geo data from the provided file (the geocoder function gives a timeout)
geodf = pd.read_csv("https://cocl.us/Geospatial_data")
geodf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [36]:
#Merging the two dataframes
df2 = pd.merge(dfgrouped, geodf, how='left', left_on=['Postcode'], right_on=['Postal Code'])
df2.drop('Postal Code', axis = 1, inplace = True)

In [37]:
df2.sample(3)

,Borough,Postcode,Neighbourhood,Latitude,Longitude
26,Downtown Toronto,M6G,Christie,43.669542,-79.422564
101,York,M6N,"The Junction North, Runnymede",43.673185,-79.487262
73,North York,M9M,"Emery, Humberlea",43.724766,-79.532242


In [38]:
df2.shape

(103, 5)